In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# 数据预处理
def preprocess_data(train, test, sequence_length):
    train_labels = train['实际功率']
    train_features = train.drop(columns=['实际功率'])
    test_labels = test['实际功率']
    test_features = test.drop(columns=['实际功率'])
    
    train_labels = np.array(train_labels).reshape(-1, 1)
    test_labels = np.array(test_labels).reshape(-1, 1)
    
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    train_features = scaler1.fit_transform(train_features)
    test_features = scaler1.transform(test_features)
    
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    train_labels = scaler2.fit_transform(train_labels)
    test_labels = scaler2.transform(test_labels)
    
    X_train, y_train = [], []
    X_test, y_test = [], []
    for i in range(len(train) - sequence_length):
        X_train.append(train_features[i:i + sequence_length])  # 特征
        y_train.append(train_labels[i + sequence_length - 1])  # 目标值（功率输出）
        
    for i in range(len(test) - sequence_length):
        X_test.append(test_features[i:i + sequence_length])  # 特征
        y_test.append(test_labels[i + sequence_length - 1])  # 目标值（功率输出）
        
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    return X_train, y_train, X_test, y_test, scaler2

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

# 定义 CNN-LSTM 模型
class CNNLSTMModel(nn.Module):
    def __init__(self, input_channels,middle_channels1, middle_channels2, out_channels, hidden_size1, hidden_size2, hidden_size3):
        super(CNNLSTMModel, self).__init__()
        self.conv1 = ConvBlock(in_channels=input_channels, out_channels = middle_channels1)
        self.conv2 = ConvBlock(in_channels=middle_channels1, out_channels = middle_channels2)
        self.conv3 = ConvBlock(in_channels=middle_channels2, out_channels = out_channels)
        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.lstm1 = nn.LSTM(input_size=32, hidden_size=hidden_size1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=hidden_size1, hidden_size=hidden_size2, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=hidden_size2, hidden_size=hidden_size3, batch_first=True)
        self.fc1 = nn.Linear(hidden_size3, 100)
        self.fc2 = nn.Linear(100, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 1)  # 调整维度以适应卷积层的输入要求
        # convblocks
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # maxpooling
        x = self.pooling(x)
        # # flatten
        x = x.permute(0, 2, 1)  # 调整维度以适应 LSTM 的输入要求
        # lstm1
        h0 = torch.zeros(self.lstm1.num_layers, x.size(0), self.lstm1.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm1.num_layers, x.size(0), self.lstm1.hidden_size).to(x.device)
        x, _ = self.lstm1(x, (h0, c0))
        x = self.relu(x)
        # lstm2
        h0 = torch.zeros(self.lstm2.num_layers, x.size(0), self.lstm2.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm2.num_layers, x.size(0), self.lstm2.hidden_size).to(x.device)
        x, _ = self.lstm2(x, (h0, c0))
        x = self.relu(x)
        # lstm3
        h0 = torch.zeros(self.lstm3.num_layers, x.size(0), self.lstm3.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm3.num_layers, x.size(0), self.lstm3.hidden_size).to(x.device)
        out, _ = self.lstm3(x, (h0, c0))
        out = self.relu(out[:, -1, :])
        # fc
        out = self.fc1(out)  # 取 LSTM 的最后一个时间步的输出
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)  # 如果使用GPU
torch.cuda.manual_seed_all(seed)  # 如果使用多个GPU
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
i=1
train = pd.read_csv('../dataset/prepared_dataset/train_{i}.csv'.format(i=i))
test = pd.read_csv('../dataset/prepared_dataset/test_{i}.csv'.format(i=i))

In [5]:
more_train = pd.read_csv('../dataset/final_dataset/train_{i}.csv'.format(i=i))
more_test = pd.read_csv('../dataset/final_dataset/test_{i}.csv'.format(i=i))
train = pd.concat([train,  (more_train['pred_power_effi'] + more_train['pred_power_cell']) / 2], axis=1)
test = pd.concat([test, (more_test['pred_power_effi'] + more_test['pred_power_cell'] ) / 2], axis=1)

In [6]:
print(train.shape, test.shape)

(53294, 170) (13372, 170)


In [7]:
test = test.fillna(0)
train = train.fillna(0)

In [ ]:
sequence_length = 24 * 4 # 假设使用 24 小时的历史数据进行预测
if i == 1:
    lr = 0.00075
else:
    lr = 0.0005
X_train, y_train, X_test, y_test, scaler = preprocess_data(train, test, sequence_length)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Training on {device}')

# 转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# 创建 DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型、损失函数和优化器
input_channels = X_train.shape[2] # 输入特征的数量
middle_channels1 = 16
middle_channels2 = 32
out_channels = 64
hidden_size1 = 16
hidden_size2 = 32
hidden_size3 = 64

model = CNNLSTMModel(input_channels = input_channels,middle_channels1 = middle_channels1, middle_channels2 = middle_channels2, 
                     out_channels = out_channels, hidden_size1 = hidden_size1, hidden_size2 = hidden_size2,hidden_size3 = hidden_size3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

Training on cuda


In [ ]:
# 初始化早停参数
early_stop_counter = 0
early_stop_threshold = 5
min_val_loss = np.inf

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # if batch_idx % 10 == 0:
        #     print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # 在验证集上评估模型
    model.eval()
    val_losses = []
    with torch.no_grad():
        for data, target in test_loader:  # 假设你有一个验证集加载器val_loader
            output = model(data)
            val_loss = criterion(output, target)
            val_losses.append(val_loss.item())
    avg_val_loss = np.mean(val_losses)
    
    print(f'Now at epoch {epoch+1}')
    # 检查早停条件
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        early_stop_counter = 0  # 重置计数器
        # 可以选择在这里保存模型的权重
        print(f'Saved at epoch {epoch+1}')
        torch.save(model.state_dict(), '../weights_cnn_lstm_combine/cnn_lstm_{i}.pth'.format(i=i))
    else:
        early_stop_counter += 1
    if early_stop_counter >= early_stop_threshold:
        print(f'Early stopping at epoch {epoch+1}')
        break

Now at epoch 1
Saved at epoch 1
Now at epoch 2
Saved at epoch 2
Now at epoch 3
Now at epoch 4
Now at epoch 5
Now at epoch 6
Now at epoch 7
Early stopping at epoch 7


In [ ]:
# 测试模型
state_dict = torch.load('../weights_cnn_lstm_combine/cnn_lstm_{i}.pth'.format(i=i))
model = CNNLSTMModel(input_channels = input_channels,middle_channels1 = middle_channels1, middle_channels2 = middle_channels2, 
                     out_channels = out_channels, hidden_size1 = hidden_size1, hidden_size2 = hidden_size2,hidden_size3 = hidden_size3).to(device)
model.load_state_dict(state_dict)
model.eval()
predictions = []
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        predictions.extend(output.cpu().numpy())

# 反归一化预测结果
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

In [12]:
# 评估模型性能
mse = mean_squared_error(y_test_actual, predictions)
mae = mean_absolute_error(y_test_actual, predictions)
rmse = rmse(y_test_actual, predictions)
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

MSE: 1.0893, MAE: 0.5803, RMSE: 1.0437
